# The Below one got 90% accuracy, with ResNet18

#Training Procedure**
- **Contrastive Learning Stage**:
  - The SimCLR model is trained using the contrastive loss on augmented image pairs.
  - Key hyperparameters:
    - Batch size: `512`
    - Learning rate: `1e-4`
    - Temperature: `0.5`
  - The Adam optimizer is used with a `StepLR` scheduler for learning rate decay.
- **Training Outputs**:
  - Loss values are logged for each epoch to monitor the learning process.

---

## Downstream Task: Classification**
- **Linear Evaluation Protocol**:
  - The pre-trained encoder is fine-tuned with a classification head for supervised learning on the labeled dataset.
  - Classification head:
    - A single linear layer mapping encoder outputs (`512`) to the number of classes.
  - Fine-tuning:
    - Encoder weights are updated at a slower learning rate (`1e-5`) than the classification head (`1e-3`).
    - Cross-entropy loss is used for optimization.
- **Performance Evaluation**:
  - Test accuracy is computed on the held-out test set to measure the model's performance.






In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import matplotlib.pyplot as plt

# Load and preprocess the dataset
train_images_path = "brain_train_image_final.npy"
train_labels_path = "brain_train_label.npy"
test_images_path = "brain_test_image_final.npy"
test_labels_path = "brain_test_label.npy"

# Load the data
final_X_train_modified = np.load(train_images_path)[:, 1, :, :]
final_X_test_modified = np.load(test_images_path)[:, 1, :, :]
train_labels = np.load(train_labels_path)
test_labels = np.load(test_labels_path)

# Normalize and Resize Images using Pillow
def normalize_and_resize(images, target_size=(224, 224)):
    resized_images = []
    for img in images:
        img = Image.fromarray((img * 255).astype(np.uint8))
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
        resized_images.append(np.array(img_resized) / 255.0)
    return np.array(resized_images)

final_X_train_resized = normalize_and_resize(final_X_train_modified)
final_X_test_resized = normalize_and_resize(final_X_test_modified)

# Define SimCLR Augmentation Transform
transform_simclr = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Custom Dataset for SimCLR
class SimCLRDataset(Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img_1 = self.transform(img)
        img_2 = self.transform(img)
        return img_1, img_2

train_dataset = SimCLRDataset(final_X_train_resized, transform_simclr)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# Define SimCLR Model
class SimCLR(nn.Module):
    def __init__(self, base_encoder, projection_dim):
        super(SimCLR, self).__init__()
        self.encoder = base_encoder
        self.projector = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, projection_dim)
        )

    def forward(self, x):
        h = self.encoder(x)
        z = self.projector(h)
        return z

# Define NT-Xent Loss
class NTXentLoss(nn.Module):
    def __init__(self, batch_size, temperature):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(reduction="sum")

    def forward(self, z_i, z_j):
        N = z_i.size(0) + z_j.size(0)
        z = torch.cat((z_i, z_j), dim=0)
        sim = torch.matmul(z, z.T) / self.temperature
        mask = ~torch.eye(N, dtype=torch.bool, device=z.device)

        positives = torch.cat([
            torch.diag(sim, z_i.size(0)),
            torch.diag(sim, -z_i.size(0))
        ])

        negatives = sim[mask].view(N, -1)
        logits = torch.cat((positives.unsqueeze(1), negatives), dim=1)
        labels = torch.zeros(N, dtype=torch.long, device=z.device)
        loss = self.criterion(logits, labels) / N
        return loss

# Initialize ResNet-18 Encoder
base_encoder = resnet18(pretrained=True)
base_encoder.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
base_encoder.fc = nn.Identity()

# Initialize SimCLR Model
model = SimCLR(base_encoder, projection_dim=128).to("cuda")
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = NTXentLoss(batch_size=512, temperature=0.5)

# Train SimCLR Model
for epoch in range(100):
    total_loss = 0
    model.train()
    for img_1, img_2 in train_loader:
        img_1, img_2 = img_1.to("cuda"), img_2.to("cuda")
        z_i = model(img_1)
        z_j = model(img_2)

        loss = criterion(z_i, z_j)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}")



# Define Dataset for Classification
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Initialize Training and Test Dataset and DataLoader
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = TestDataset(final_X_train_resized, train_labels, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = TestDataset(final_X_test_resized, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Add Classification Head
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

classification_head = ClassificationHead(input_dim=512, num_classes=len(np.unique(train_labels))).to("cuda")
optimizer_cls = optim.Adam([
    {"params": model.encoder.parameters(), "lr": 1e-5},
    {"params": classification_head.parameters(), "lr": 1e-3},
])
scheduler_cls = StepLR(optimizer_cls, step_size=10, gamma=0.5)
criterion_cls = nn.CrossEntropyLoss()

# Fine-tune Classification Head
for epoch in range(100):
    model.encoder.train()
    classification_head.train()
    total_loss = 0
    correct = 0
    for img, label in DataLoader(train_dataset, batch_size=128, shuffle=True):
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        loss = criterion_cls(logits, label)

        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()

        total_loss += loss.item()
        correct += (logits.argmax(dim=1) == label).sum().item()

    accuracy = correct / len(train_labels)
    scheduler_cls.step()
    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

# Evaluate on Test Dataset
classification_head.eval()
correct = 0
with torch.no_grad():
    for img, label in DataLoader(test_dataset, batch_size=128, shuffle=False):
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        correct += (logits.argmax(dim=1) == label).sum().item()

test_accuracy = correct / len(test_labels)
print(f"Test Accuracy: {test_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/100], Loss: 8.1232
Epoch [2/100], Loss: 6.6977
Epoch [3/100], Loss: 6.5628
Epoch [4/100], Loss: 6.4204
Epoch [5/100], Loss: 6.2939
Epoch [6/100], Loss: 6.2331
Epoch [7/100], Loss: 6.1769
Epoch [8/100], Loss: 6.0556
Epoch [9/100], Loss: 5.9819
Epoch [10/100], Loss: 5.8945
Epoch [11/100], Loss: 5.7674
Epoch [12/100], Loss: 5.6089
Epoch [13/100], Loss: 5.5685
Epoch [14/100], Loss: 5.4512
Epoch [15/100], Loss: 5.3110
Epoch [16/100], Loss: 5.1675
Epoch [17/100], Loss: 5.1007
Epoch [18/100], Loss: 4.9024
Epoch [19/100], Loss: 4.8179
Epoch [20/100], Loss: 4.7292
Epoch [21/100], Loss: 4.6493
Epoch [22/100], Loss: 4.5762
Epoch [23/100], Loss: 4.4276
Epoch [24/100], Loss: 4.3101
Epoch [25/100], Loss: 4.2424
Epoch [26/100], Loss: 4.1504
Epoch [27/100], Loss: 4.0430
Epoch [28/100], Loss: 3.9242
Epoch [29/100], Loss: 3.8458
Epoch [30/100], Loss: 3.7267
Epoch [31/100], Loss: 3.8131
Epoch [32/100], Loss: 3.7184
Epoch [33/100], Loss: 3.6482
Epoch [34/100], Loss: 3.5053
Epoch [35/100], Loss: 3

# The below one got 85% accuracy, with ResNet34

# SimCLR Implementation with ResNet-34

## Training Procedure
- **Contrastive Learning Stage**:
  - The SimCLR model is trained on the augmented pairs using contrastive loss.
  - Key hyperparameters:
    - Batch size: `512`
    - Learning rate: `3e-4`
    - Temperature: `0.5`
  - Optimizer: Adam
  - Scheduler: `CosineAnnealingLR` with `T_max=100` for smooth learning rate decay.

---

## Downstream Task: Classification
- **Linear Evaluation Protocol**:
  - The pre-trained encoder is fine-tuned with a linear classification head for the labeled dataset.
  - Classification head:
    - A single `Linear(512, num_classes)` layer maps encoder features to class logits.
  - Fine-tuning strategy:
    - Encoder weights updated at a slower learning rate (`1e-5`) compared to the classification head (`3e-4`).
    - Cross-entropy loss is used for optimization.







In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet34, ResNet34_Weights
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image

# Load and preprocess the dataset
train_images_path = "brain_train_image_final.npy"
train_labels_path = "brain_train_label.npy"
test_images_path = "brain_test_image_final.npy"
test_labels_path = "brain_test_label.npy"

# Load the data
final_X_train_modified = np.load(train_images_path)[:, 1, :, :]
final_X_test_modified = np.load(test_images_path)[:, 1, :, :]
train_labels = np.load(train_labels_path)
test_labels = np.load(test_labels_path)

# Normalize and Resize Images using Pillow
def normalize_and_resize(images, target_size=(224, 224)):
    resized_images = []
    for img in images:
        img = Image.fromarray((img * 255).astype(np.uint8))
        img_resized = img.resize(target_size, Image.Resampling.LANCZOS)
        resized_images.append(np.array(img_resized) / 255.0)
    return np.array(resized_images)

final_X_train_resized = normalize_and_resize(final_X_train_modified)
final_X_test_resized = normalize_and_resize(final_X_test_modified)

# Define SimCLR Augmentation Transform
transform_simclr = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=224, scale=(0.08, 1.0), ratio=(3/4, 4/3)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.8, contrast=0.8, saturation=0.8, hue=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=23, sigma=(0.1, 2.0))], p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Custom Dataset for SimCLR
class SimCLRDataset(Dataset):
    def __init__(self, images, transform):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        img_1 = self.transform(img)
        img_2 = self.transform(img)
        return img_1, img_2

train_dataset = SimCLRDataset(final_X_train_resized, transform_simclr)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

# Define SimCLR Model
class SimCLR(nn.Module):
    def __init__(self, base_encoder, projection_dim):
        super(SimCLR, self).__init__()
        self.encoder = base_encoder
        self.projector = nn.Sequential(
            nn.Linear(512, 512),  # Adjusted for ResNet-34 output
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )

    def forward(self, x):
        h = self.encoder(x)
        z = self.projector(h)
        return z

# Define NT-Xent Loss
class NTXentLoss(nn.Module):
    def __init__(self, temperature):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(reduction="mean")

    def forward(self, z_i, z_j):
        N = z_i.size(0) + z_j.size(0)
        z = torch.cat((z_i, z_j), dim=0)
        sim = torch.mm(z, z.T) / self.temperature
        sim = torch.nn.functional.softmax(sim, dim=1)

        labels = torch.cat([
            torch.arange(z_i.size(0), device=z.device),
            torch.arange(z_j.size(0), device=z.device)
        ])
        loss = self.criterion(sim, labels)
        return loss

# Initialize ResNet-34 Encoder with updated weights argument
base_encoder = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
base_encoder.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
base_encoder.fc = nn.Identity()

# Initialize SimCLR Model
model = SimCLR(base_encoder, projection_dim=128).to("cuda")
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = NTXentLoss(temperature=0.5)
scheduler = CosineAnnealingLR(optimizer, T_max=100)

# Train SimCLR Model
for epoch in range(100):
    total_loss = 0
    model.train()
    for img_1, img_2 in train_loader:
        img_1, img_2 = img_1.to("cuda"), img_2.to("cuda")
        z_i = model(img_1)
        z_j = model(img_2)

        loss = criterion(z_i, z_j)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}")

# Define Dataset for Classification
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Initialize Training and Test Dataset and DataLoader
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = TestDataset(final_X_train_resized, train_labels, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = TestDataset(final_X_test_resized, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Add Classification Head
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

classification_head = ClassificationHead(input_dim=512, num_classes=len(np.unique(train_labels))).to("cuda")
optimizer_cls = optim.Adam([
    {"params": model.encoder.parameters(), "lr": 1e-5},
    {"params": classification_head.parameters(), "lr": 3e-4},
])
criterion_cls = nn.CrossEntropyLoss()

# Fine-tune Classification Head
for epoch in range(100):
    model.encoder.train()
    classification_head.train()
    total_loss = 0
    correct = 0
    for img, label in train_loader:
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        loss = criterion_cls(logits, label)

        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()

        total_loss += loss.item()
        correct += (logits.argmax(dim=1) == label).sum().item()

    accuracy = correct / len(train_labels)
    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

# Evaluate on Test Dataset
classification_head.eval()
correct = 0
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        correct += (logits.argmax(dim=1) == label).sum().item()

test_accuracy = correct / len(test_labels)
print(f"Test Accuracy: {test_accuracy*100:.4f}")


Epoch [1/100], Loss: 6.3819
Epoch [2/100], Loss: 6.1793
Epoch [3/100], Loss: 6.1024
Epoch [4/100], Loss: 6.0910
Epoch [5/100], Loss: 6.0812
Epoch [6/100], Loss: 6.0800
Epoch [7/100], Loss: 6.0776
Epoch [8/100], Loss: 6.0784
Epoch [9/100], Loss: 6.0793
Epoch [10/100], Loss: 6.0791
Epoch [11/100], Loss: 6.0776
Epoch [12/100], Loss: 6.0777
Epoch [13/100], Loss: 6.0779
Epoch [14/100], Loss: 6.0781
Epoch [15/100], Loss: 6.0788
Epoch [16/100], Loss: 6.0781
Epoch [17/100], Loss: 6.0792
Epoch [18/100], Loss: 6.0778
Epoch [19/100], Loss: 6.0781
Epoch [20/100], Loss: 6.0769
Epoch [21/100], Loss: 6.0781
Epoch [22/100], Loss: 6.0765
Epoch [23/100], Loss: 6.0763
Epoch [24/100], Loss: 6.0758
Epoch [25/100], Loss: 6.0762
Epoch [26/100], Loss: 6.0758
Epoch [27/100], Loss: 6.0760
Epoch [28/100], Loss: 6.0758
Epoch [29/100], Loss: 6.0762
Epoch [30/100], Loss: 6.0770
Epoch [31/100], Loss: 6.0752
Epoch [32/100], Loss: 6.0767
Epoch [33/100], Loss: 6.0752
Epoch [34/100], Loss: 6.0766
Epoch [35/100], Loss: 6

# The below one is used the previous SimCLR but has different fine-tuning epoches=200 for testing. the accuracy is 89%


- **Batch Size**:
  - `512` for both training and testing.
- **Shuffling**:
  - Training set is shuffled to enhance generalization during training.
  - Testing set is not shuffled, ensuring deterministic evaluation.

---

## **4. Fine-Tuning with a Classification Head**
- **Classification Head**:
  - A single linear layer (`Linear(512, num_classes)`) maps encoder features to the output class logits.
- **Fine-Tuning Procedure**:
  - The SimCLR encoder is fine-tuned along with the classification head.
  - Hyperparameters:
    - Learning rate for encoder: `1e-5`
    - Learning rate for classification head: `3e-4`
    - Loss function: `CrossEntropyLoss`
    - Epochs: `200`
  - Optimizer: Adam optimizes both the encoder and classification head weights.
- **Training Outputs**:
  - The average training loss and accuracy are logged for each epoch.





In [ ]:
# Define Dataset for Classification
class TestDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Initialize Training and Test Dataset and DataLoader
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = TestDataset(final_X_train_resized, train_labels, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)

test_dataset = TestDataset(final_X_test_resized, test_labels, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Add Classification Head
class ClassificationHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassificationHead, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

classification_head = ClassificationHead(input_dim=512, num_classes=len(np.unique(train_labels))).to("cuda")
optimizer_cls = optim.Adam([
    {"params": model.encoder.parameters(), "lr": 1e-5},
    {"params": classification_head.parameters(), "lr": 3e-4},
])
criterion_cls = nn.CrossEntropyLoss()

# Fine-tune Classification Head
for epoch in range(200):
    model.encoder.train()
    classification_head.train()
    total_loss = 0
    correct = 0
    for img, label in train_loader:
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        loss = criterion_cls(logits, label)

        optimizer_cls.zero_grad()
        loss.backward()
        optimizer_cls.step()

        total_loss += loss.item()
        correct += (logits.argmax(dim=1) == label).sum().item()

    accuracy = correct / len(train_labels)
    print(f"Epoch [{epoch+1}/100], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}")

# Evaluate on Test Dataset
classification_head.eval()
correct = 0
with torch.no_grad():
    for img, label in test_loader:
        img, label = img.to("cuda"), label.to("cuda")
        features = model.encoder(img)
        logits = classification_head(features)
        correct += (logits.argmax(dim=1) == label).sum().item()

test_accuracy = correct / len(test_labels)
print(f"Test Accuracy: {test_accuracy*100:.4f}")


Epoch [1/100], Loss: 1.1620, Accuracy: 0.3193
Epoch [2/100], Loss: 0.6423, Accuracy: 0.8497
Epoch [3/100], Loss: 0.3671, Accuracy: 0.9897
Epoch [4/100], Loss: 0.2183, Accuracy: 0.9976
Epoch [5/100], Loss: 0.1332, Accuracy: 0.9994
Epoch [6/100], Loss: 0.0877, Accuracy: 0.9994
Epoch [7/100], Loss: 0.0580, Accuracy: 0.9994
Epoch [8/100], Loss: 0.0429, Accuracy: 0.9994
Epoch [9/100], Loss: 0.0326, Accuracy: 1.0000
Epoch [10/100], Loss: 0.0248, Accuracy: 1.0000
Epoch [11/100], Loss: 0.0214, Accuracy: 1.0000
Epoch [12/100], Loss: 0.0174, Accuracy: 1.0000
Epoch [13/100], Loss: 0.0147, Accuracy: 1.0000
Epoch [14/100], Loss: 0.0130, Accuracy: 1.0000
Epoch [15/100], Loss: 0.0127, Accuracy: 1.0000
Epoch [16/100], Loss: 0.0102, Accuracy: 1.0000
Epoch [17/100], Loss: 0.0101, Accuracy: 1.0000
Epoch [18/100], Loss: 0.0088, Accuracy: 1.0000
Epoch [19/100], Loss: 0.0082, Accuracy: 1.0000
Epoch [20/100], Loss: 0.0072, Accuracy: 1.0000
Epoch [21/100], Loss: 0.0086, Accuracy: 1.0000
Epoch [22/100], Loss: 